In [ ]:
from pavlov import *
monitor()

In [ ]:
from pavlov import stats
stats.plot(['2021-01-05 18-37-29 gentle-mouth 7x7', '*7x7-no-league*', -1], 'elo-mohex', 'μ', fill=True, skip=1)

In [ ]:
runs.pandas().query('boardsize==7')